# Earliest Calibration Date by System

### Imports
Import Python modules for executing the notebook. Pandas is used for building and handling dataframes. Scrapbook is used for recording data for the Notebook Execution Service.

In [1]:
import pandas as pd
import scrapbook as sb

from datetime import datetime

from systemlink.clients.niapm import AssetsApi, QueryAssetsRequest

### Parameters
- `group_by`: The property by which data is grouped. This is unused in this simple example, but for the data to be available as a column in the Systems Grid, we must at minimum support 'System' here. Other data groupings could be used for multi-purpose notebooks.

Parameters are also listed in the metadata for the parameters cell, along with their default values.
To see the metadata, select the code cell below and click the wrench icon in the far left panel.

In [2]:
group_by = 'System'

### Query for assets
Query the Asset Service for all assets that support external calibration then create a list of all assets with Calibration and Location data.

In [3]:
assets_api = AssetsApi()
query_result = await assets_api.query_assets(query_assets=QueryAssetsRequest(filter='SupportsExternalCalibration = true'))
results = [asset for asset in query_result.assets if asset.external_calibration is not None and asset.location.minion_id != '']

### Calculate earliest asset calibration date for each system and create pandas dataframe

In [4]:
earliest_cal = {}

# create an entry in in the earliest_cal dict for each asset with the minion_id as the key. On collision, take the earlier date value.
for asset in results:
    if asset.location.minion_id not in earliest_cal:
        earliest_cal[asset.location.minion_id] = asset.external_calibration.next_recommended_date
    else:
        earliest_cal[asset.location.minion_id] = earliest_cal[asset.location.minion_id] if earliest_cal[asset.location.minion_id] < asset.external_calibration.next_recommended_date else asset.external_calibration.next_recommended_date
        
# Create a dataframe from the earliest_cal dict and display results here
df = pd.DataFrame.from_dict(earliest_cal, orient='index', columns=['Earliest Calibration Date'])
df

,Earliest Calibration Date
NI_cRIO-9042--SN-01E10AB8--MAC-00-80-2F-24-FB-CE,2022-06-19 17:00:00+00:00
Precision_3630_Tower--SN-516VCV2--MAC-B8-85-84-BB-67-ED,2021-10-15 17:00:00+00:00
NI_cRIO-9042--SN-01E10AB4--MAC-00-80-2F-24-FB-C2,2022-06-19 17:00:00+00:00
NI_PXIe-8135_Embedded_Controller--MAC-00-80-2F-24-66-EB,2019-10-19 06:37:14+00:00
NI_PXIe-8135_Embedded_Controller--MAC-00-80-2F-26-2A-6D,2020-03-14 13:54:16+00:00
Precision_3630_Tower--SN-518TCV2--MAC-B8-85-84-BB-C1-67,2016-04-05 13:55:27+00:00
NI_cRIO-9042--SN-01E10AC1--MAC-00-80-2F-24-FB-E9,2022-06-19 17:00:00+00:00
NI_PXIe-8880--SN-030D0A62--MAC-00-80-2F-23-F1-D5,2013-12-04 06:00:00+00:00
Battery_Tester--SN-123456--MAC-34-17-EB-E2-43-C0,2007-12-04 16:27:06+00:00
MS-7C37--MAC-2C-F0-5D-28-17-81,2014-08-23 09:31:38+00:00


### Convert the dataframe to the SystemLink reports output format

In [5]:
df_dict = {
    'columns': ['minion id', 'earliest calibration date'],
    'values': df.reset_index().values.tolist()
}

result = [{
    "display_name": "Earliest calibration date by system",
    "id": 'earliest_cal_by_system',
    "type": 'data_frame',
    "data": df_dict
}]

In [6]:
sb.glue('result', result)

### View the output of this report in the Systems Grid (SystemLink 2021 R2 and later)
1. On your SystemLink Server, ensure the NI SystemLink Server - JupyterHub Module is installed
1. In SystemLink, navigate to Utilities -> Jupyter, and create a folder named 'reports' if it does not already exist
1. Upload this notebook to the reports folder
1. From the Systems page, press the edit grid button in the upper-right section of the grid area
1. Press the '+ ADD' button to add a new column, and select 'Notebook' as the data source
1. Select this report in the Path field ([your_username]/reports/EarliestCalBySystem.ipynb)
1. In the Output field, select 'Earliest calibration date by system'
1. Select an appropriate Update interval for your needs (5 min to 24 hours)
1. Enter an appropriate Column name (ex. 'Next Cal Date') and press Done
